<a href="https://colab.research.google.com/github/vfulle/FMF-VSC/blob/master/Copy_of_causal_inference_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Causal Inference in Neuroimaging Tutorial #
## Pattern Recognition in Neuroimaging (PRNI) 2020 Virtual Summer School ##
### Lecturer: Moritz Grosse-Wentrup, Research Group Neuroinformatics, University of Vienna ###
### Tutor: [Alex Markham](https://causal.dev), Research Group Neuroinformatics, University of Vienna ###

-------------------------------------------------------------------------------

  * [1. Introduction](#1-introduction)
  * [2. Data sets](#2-data-sets)
  * [3. Statistical tests](#3-statistical-tests)
  * [4. Causal interpretation of variables](#4-causal-interpretation-of-variables)
  * [5. Further exercises](#5-further-exercises)
  * [6. References](#6-references)
  
-------------------------------------------------------------------------------

### 1. Introduction ###

In this tutorial, you will be given artificial data on which you can practice applying statistical tests (using Python) and making *valid* causal claims (using the causal interpretation rules in [[Weichwald et al. 2015]](#weichwald)).

### 2. Data sets ###

There are two main data sets, one simulating a stimulus-based experimental paradigm and one for response-based, as well as two additional data sets which will be used in [6. Further exercises](#6-further-exercises). 
The main stimulus-based data set consists of 1000 samples of a single stimulus variable and 6 features (for example, these could be 6 regions of interest in fMRI data), all of which are connected by an unknown causal structure. 
The main response-based data set is similar, but with a single response variable instead of a stimulus variable, and a different causal structure. 
The data sets are saved as text files named [`stimulus_based.txt`](https://gitlab.cs.univie.ac.at/markhama94cs/prni_causality_tutorial/-/raw/master/stimulus_based.txt?inline=false) and [`response_based.txt`](https://gitlab.cs.univie.ac.at/markhama94cs/prni_causality_tutorial/-/raw/master/response_based.txt?inline=false) respectively, and the first row of each has the names of the corresponding variables.

You can extract the variable names from the first line of the text file with the following code:

In [ ]:
with open('stimulus_based.txt') as f:
	header = f.readline()
var_names = header.rstrip('\n').split(' ')


**Tasks**
  *  Try `len(var_names)` to see how many variables there are, and `print(var_names)` to see all their names.
  *  Use the [NumPy `loadtxt()`](https://numpy.org/doc/stable/reference/generated/numpy.loadtxt.html) function, with the the `skiprows=1` argument (since you already loaded that row as `var_names`), to load the data from the stimulus-based experiment. 
  This will load the data as a 2-d numpy array (so think of it as a matrix).
  *  Use the [Numpy `shape()`](https://numpy.org/doc/stable/reference/generated/numpy.shape.html) function to print the shape
  *  Print the 0th column of the data. (*hint*: see NumPy [documentation on indexing](https://numpy.org/doc/stable/reference/arrays.indexing.html))
  *  Now print all but the 0th column of the data.
  *  Repeat the above tasks for the data from the response-based experiment.

In [ ]:
# Complete tasks here



### 3. Statistical tests ###

For each observed variable, you will now perform statistical tests to estimate their unconditional and conditional dependence with the stimulus.
(*Note*: independence here means the binary, symmetric *relation* of [probabilistic independence](https://en.wikipedia.org/wiki/Independence_(probability_theory)), and two variables are dependent if and only if they are not independent. 
This is different from the meaning of the *adjectives* [dependent and independent](https://en.wikipedia.org/wiki/Dependent_and_independent_variables) that is commonly used in the experimental sciences.)

To estimate unconditional dependencies, you will calculate the Pearson correlation coefficients and corresponding p-values using [SciPy `stats.pearsonr()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html) function. 
You will need to call this function six times (to find the correlation between each of the six observed variables and the stimulus).

To estimate conditional dependencies, you will calculate the partial correlation coefficients (via ordinary least squares linear regression) and p-values using the [statsmodels `OLS()`](https://www.statsmodels.org/devel/generated/statsmodels.regression.linear_model.OLS.html#statsmodels.regression.linear_model.OLS) class and its `fit()` method. 
You will only need to fit the model once (because solving the linear regression problem returns all six coefficients), and then can view the both the coefficients and p-values (and many other statistics) with the `results.summary()` method or individually using the `results.params` attribute and `results.pvalues` property, respectively. 
Importantly, you must manually add a column of ones to the data set before finding the regression model. 
This corresponds to the "bias" or "intercept" term in the regression problem. 
You may find the NumPy functions [`ones()`](https://numpy.org/doc/stable/reference/generated/numpy.ones.html) and [`hstack()`](https://numpy.org/doc/stable/reference/generated/numpy.hstack.html) helpful for this.

In both cases, we are interested in learning the dependencies (or conditional dependencies) between random variables, but we only have access to correlations (or partial correlations) computed on a sample from these random variables.
(*Note*: This is an instance of one of the most fundamental limitations in scientific reasoning, the [Problem of Induction](https://en.wikipedia.org/wiki/Problem_of_induction).) 
A common method for using the correlations to reason about the underlying dependencies is to perform statistical hypothesis testing, which is why the python functions specified above also return p-values. In each test, the null hypothesis is that the two random variables are independent (e.g., S \indep X_1). 
The corresponding p-value is the probability that, assuming the null hypothesis is true, a sample from these independent random variables would have a correlation at least as extreme as the one computed (and extreme in this case means far from 0, because 0 correlation implies linear independence). 
If a p-value is sufficiently small (meaning a sample from independent random variables is unlikely to have such a high correlation), then the null hypothesis is rejected (so the variables are taken to be dependent). 
For a more detailed but still concise overview, see Chapter 10: *Hypothesis Testing and p-values* in [[Wasserman 2013]](#wasserman).

*Note:* The use of Pearson correlation and partial correlation computed by linear regression to estimate un/conditional independencies here is only valid because the data set is artificial and the underlying functional causal model is linear. 
In practice, you would need to check whether your data is linear and use nonlinear measures of dependence if not, such as the [distance correlation](https://dcor.readthedocs.io/en/latest/) or [Hilbert-Schmidt independence criterion (HSIC)](https://github.com/amber0309/HSIC).

**Tasks**

*  Perform the necessary tests in Python and fill out the correlation (and partial correlation) and p-value columns in the following table for the stimulus-based experiment data set. 
*  Using a significance level of `alpha = 0.05`, fill out the dependence (and conditional dependence) columns, i.e., when the p-value is less than alpha, reject the null hypothesis of independence. The notation `X_-i` means all variables in `{X_1, ..., X_6}` other than `X_i`.
(*Note*: Because this is an artificial data set, I have ensured this significance level produces correct results.) 
However, justified use in practice would require carefully considering what significance level is right for that application and being aware of its implications and limitations.)
*  Make an analogous table for the response-based experiment data set, perform the corresponding tests, and fill it out too.

| `X_i` | | corr(`X_i`, S) | p-value | `X_i` \dep S? | | par_corr(`X_i`, S) | p-value | `X_i` \dep S \given `X_-i`? |
| :---- |-| :------------: | :-----: | :-----------: |-|:-----------------: | :-----: | :-------------------------: |
| `X_1` | |                |         |               | |                    |         |                             |
| `X_2` | |                |         |               | |                    |         |                             |
| `X_3` | |                |         |               | |                    |         |                             |
| `X_4` | |                |         |               | |                    |         |                             |
| `X_5` | |                |         |               | |                    |         |                             |
| `X_6` | |                |         |               | |                    |         |                             |


In [ ]:
# Complete tasks here


### 4. Causal interpretation of variables ###

A variable `X_i` is encoding relevant if and only if `X_i` \dep C (whether the variable `C` corresponds to a stimulus S or a response R), and decoding relevant if and only if `X_i` \dep C \given `X_-i`.
The following table (from [[Weichwald et al. 2015]](#weichwald)) shows which causal interpretations are implied by which combinations of encoding and decoding relevance, for stimulus-based and response-based experimental settings. 

#### Table 4.1.: Causal Interpretation Rules  ####
  ![causal interpretation rules](https://gitlab.cs.univie.ac.at/markhama94cs/prni_causality_tutorial/-/raw/master/causal_interpretation_rules.png)

**Tasks**

*  For the stimulus-based data set, use the estimated dependencies from [Section 3](#3-statistical-tests) to fill in the encoding relevance in the following table, and use the conditional dependencies to fill in the decoding relevance.
*  Based on these, consult [Table 4.1.](#table-4-1-causal-interpretation-rules) to fill in the last column of the table.
*  Make an analogous table for the response-based experiment data set and fill it out similarly.

| variable | encoding relevant? | decoding relevant? | causal interpretation? |
| :------- | :----------------: | :----------------: | :--------------------: |
| `X_1`    |                    |                    |                        |
| `X_2`    |                    |                    |                        |
| `X_3`    |                    |                    |                        |
| `X_4`    |                    |                    |                        |
| `X_5`    |                    |                    |                        |
| `X_6`    |                    |                    |                        |

  *  Stimulus-based:
	 *  What are the possible direct effects of S? 
	 *  Can you think of a way to tell if any of these are actually a direct effect or not? 
	 *  How might they be related to S if they aren't a direct effect? (*hint*: consider that there could be unmeasured latent variables)
  *  Response-based:
	 *  What are the possible direct causes of R? 
	 *  Can you think of a way to tell if any of these are actually a direct cause or not? 
	 *  How might they be related to R if they aren't a direct cause?


In [ ]:
# Complete tasks here


### 5. Further Exercises ###

Unless you are already experienced with Python and causal inference, you will likely not have time in the tutorial session to get to the following exercises.
They are nevertheless available for you to do on your own later to gain a deeper understanding.

#### 5.1. The larger causal model ####

Now that you have the causal interpretation for each individual variable, you can try piecing it all together to see what can be learned about the full causal structure.

**Tasks**
  *  How much of the causal model represented as a DAG can you reconstruct for each data set using only the information so far available?
  *  Can you write down a full DAG (for each setting) that is consistent with (even if not necessitated by) your interpretations from the above sections?
  *  For reconstructing a DAG, now try the following approach:

        1.  start with a fully-connected undirected graph (so (7 choose 2) = 21 edges total between the 7 variables)
		2.  remove all the edges that were ruled out in previous sections
        3.  direct all the edges you can with the findings from the previous sections

  This is called an *essential graph* and represents the *Markov equivalence class* of all DAGs that are consistent with the conditional independence relations you have estimated.
  *  Try conditioning on different subsets instead of always X_-i, i.e., rerun the linear regression model on different subsets of variables, to get more conditional independence constraints and use these to refine the essential graph found above. Compare this to the given true DAGs and notice the difficulty of causal inference, even in a small, artificial setting like this!

In [ ]:
# Complete tasks here


#### 5.1. The impact of statistical estimation errors ####

I have provided two additional data sets, saved as text files named [`unfaithful_stimulus_based.txt`](https://gitlab.cs.univie.ac.at/markhama94cs/prni_causality_tutorial/-/raw/master/unfaithful_response_based.txt?inline=false) and [`unfaithful_response_based.txt`](https://gitlab.cs.univie.ac.at/markhama94cs/prni_causality_tutorial/-/raw/master/unfaithful_response_based.txt?inline=false) respectively.
These data sets are generated from the same underlying causal models used for all the above sections, but the samples have been specially chosen so that they do not faithfully represent the generating causal models.

**Tasks**
  *  Repeat as much of all the above analyses as you would like on these two new data sets to see how much of an impact errors in statistical estimation have on the causal conclusions.
  *  Also try using different values for `alpha`. Can you find one that works better than `0.05` in overcoming some of the errors from these unfaithful samples?
  *  The value you set for `alpha` is the probability of making a Type I error. Consider also the Type II error, made with probability `beta`. Can you detect any relationship between `alpha` and `beta`, e.g., is `beta` always higher or lower, or how does `beta` change as you change `alpha`?


In [ ]:
# Complete tasks here


### 6. References ###

<a name="weichwald">[Weichwald et al. 2015]</a>: Weichwald, S., Meyer, T., Özdenizci, O., Schölkopf, B., Ball, T., & Grosse-Wentrup, M. (2015). [*Causal interpretation rules for encoding and decoding models in neuroimaging*](https://sci-hub.tw/10.1016/j.neuroimage.2015.01.036)  NeuroImage, 110, 48-59.

<a name="wasserman">[Wasserman 2013]</a>: Wasserman, Larry. [*All of statistics: a concise course in statistical inference*](https://1lib.eu/s/all%20of%20statistics%20wasserman) Springer Science & Business Media, 2013.